In [0]:
#The Generator 
import keras 
from keras import layers
import numpy as np


Using TensorFlow backend.


In [0]:
latent_dim = 32
height =32
width = 32
channels = 3


In [0]:
generator_input = keras.Input(shape=(latent_dim,))


In [0]:
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16,16,128))(x)


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
x = layers.Conv2D(256,5,padding='same')(x)
x = layers.LeakyReLU()(x)


In [0]:
x = layers.Conv2DTranspose(265,4,strides=2,padding='same')(x)
x = layers.LeakyReLU()(x)


In [0]:
x = layers.Conv2D(256,5,padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256,5,padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels,7,activation='tanh',padding='same')(x)


In [0]:
generator = keras.models.Model(generator_input,x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 265)       1085705   
__________

In [0]:
#The Discriminator
discriminator_input = layers.Input(shape=(height,width,channels))
x = layers.Conv2D(128,3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)


In [0]:
x = layers.Dropout(0.4)(x)
x = layers.Dense(1,activation='sigmoid')(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
discriminator = keras.models.Model(discriminator_input,x)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0         
__________

In [0]:
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.00008,clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')

In [0]:
#The adversarial Network
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input,gan_output)


In [0]:
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004,clipvalue=1.0,decay=1e-8)
gan.compile(optimizer=gan_optimizer,loss='binary_crossentropy')

In [0]:
#Implementing the GAN
import os 
from keras.preprocessing import image


In [0]:
(x_train,y_train),(_,_) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 38s 0us/step


In [0]:
x_train = x_train[y_train.flatten()==0]#selection of airplanes
#Normalizing the Data
x_train = x_train.reshape((x_train.shape[0],) + (height,width,channels)).astype('float32')/255

In [0]:
import os
import shutil

In [0]:
output_data = 'output_gan'

In [0]:
os.mkdir(output_data)

In [0]:
iterations = 10000
batch_size = 20
save_dir = output_data

In [0]:
start = 0

In [0]:
for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
  generated_images = generator.predict(random_latent_vectors)
  stop = start + batch_size
  real_images = x_train[start:stop]
  combined_images = np.concatenate([generated_images,real_images])
  labels = np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])
  labels += 0.5 * np.random.random(labels.shape)
  d_loss = discriminator.train_on_batch(combined_images,labels)
  random_latent_vectors = np.random.normal(size = (batch_size,latent_dim))
  misleading_targets = np.zeros((batch_size,1))
  a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets)
  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
  if start%100 == 0:
    gan.save_weights('gan.h5')
    print('discriminator loss:',d_loss)
    print('adversarial_loss:',a_loss)
    img = image.array_to_img(generated_images[0]*255.,scale=False)
    img.save(os.path.join(save_dir,'generated_frog'+str(step)+'.png'))
    img = image.array_to_img(real_images[0] * 255.,scale=False)
    img.save(os.path.join(save_dir,'real_frog'+str(step)+'.png'))

Instructions for updating:
Use tf.cast instead.
discriminator loss: 0.7247027
adversarial_loss: 0.6903143
discriminator loss: 0.73040116
adversarial_loss: 0.66016865
discriminator loss: 0.7259189
adversarial_loss: 0.6463854
discriminator loss: 0.7376262
adversarial_loss: 0.6647957
discriminator loss: 0.72286403
adversarial_loss: 0.6524652
discriminator loss: 0.72551674
adversarial_loss: 0.6520138
discriminator loss: 0.7347124
adversarial_loss: 0.67274904
discriminator loss: 0.7326789
adversarial_loss: 0.6686576
discriminator loss: 0.7307528
adversarial_loss: 0.65347534
discriminator loss: 0.73493016
adversarial_loss: 0.6611598
discriminator loss: 0.7329354
adversarial_loss: 0.6482543
discriminator loss: 0.7327161
adversarial_loss: 0.6767459
discriminator loss: 0.7194843
adversarial_loss: 0.6482684
discriminator loss: 0.71756756
adversarial_loss: 0.65370286
discriminator loss: 0.7441896
adversarial_loss: 0.62783873
discriminator loss: 0.7439485
adversarial_loss: 0.6435927
discriminator 

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
shutil.copytree('output_gan','/drive/My \Drive')

'/drive/My \\Drive'

In [0]:
!zip -r /content/output.zip /content/output_gan

  adding: content/output_gan/ (stored 0%)
  adding: content/output_gan/real_frog4459.png (stored 0%)
  adding: content/output_gan/real_frog7899.png (stored 0%)
  adding: content/output_gan/generated_frog394.png (stored 0%)
  adding: content/output_gan/real_frog7094.png (stored 0%)
  adding: content/output_gan/generated_frog2599.png (stored 0%)
  adding: content/output_gan/real_frog5044.png (stored 0%)
  adding: content/output_gan/real_frog434.png (stored 0%)
  adding: content/output_gan/generated_frog8724.png (stored 0%)
  adding: content/output_gan/generated_frog9854.png (stored 0%)
  adding: content/output_gan/real_frog8664.png (stored 0%)
  adding: content/output_gan/real_frog5294.png (stored 0%)
  adding: content/output_gan/generated_frog3084.png (stored 0%)
  adding: content/output_gan/real_frog9214.png (stored 0%)
  adding: content/output_gan/real_frog7784.png (stored 0%)
  adding: content/output_gan/real_frog2844.png (stored 0%)
  adding: content/output_gan/real_frog5819.png (st

In [0]:
from google.colab import files

In [0]:
files.download('/content/output.zip')